In [ ]:
### 通常は、よく使う下記のライブラリを最初に読み込んでおくと便利 ###
# ### 必要なパッケージ（ライブラリ）の読み込み ###
# import pandas as pd             # データフレーム型変数を取り扱えるパッケージ"pandas"を読込み（以降"pd"と略記）
# import matplotlib.pyplot as plt # グラフ描画のパッケージ"matplotlib"を読込み（以降"plt"と略記）
# import seaborn as sns           # 上記matplotlibをベースにした高機能可視化モジュール"saeborn"を読込み（以降"sns"と略記）
# import numpy as np              # 数値演算のためのパッケージ"numpy"を読込み（以降"np"と略記）

# データの読込・閲覧

## 準備（必要なパッケージ、データの読込）

In [ ]:
import pandas as pd              # データフレーム型変数を取り扱えるパッケージpandasを読込
df = pd.read_excel('ファイル名入力（Tabキーによる文字補完を活用）') #データ読込

# ※ "演習01_" まで入力し、TABキーで補完するのが効率的

## データ閲覧

In [ ]:
df

In [ ]:
df.head(3)

## 行数・列数の出力

In [ ]:
df.shape

## 要約統計量の出力

In [ ]:
df.describe()

In [ ]:
### 参考：カテゴリー値の場合 ###
df.describe(include='O')   #'Object'型の頭文字'O'

## 相関行列

In [ ]:
### 相関行列 ###
df.corr(numeric_only=True) 

# pandasの仕様変更(おそらくv2.0.0以降）により、numeric_only=Trueの指定が必須化
# 指定しないと、文字列を含む列がある場合、"could not convert string to float...." というエラーが発生
# （従来は、当該引数を指定しなくても、自動的に文字列を含む列は除外されていた）
# 
# 参考 pandasのversion確認方法：print(pd.__version__)

In [ ]:
### 相関行列のお化粧（ヒートマップ） ###

import matplotlib.pyplot as plt # グラフ描画のパッケージ"matplotlib"を読込み（以降"plt"と略記）
import seaborn as sns           # 上記matplotlibをベースにした高機能可視化モジュール"saeborn"を読込み（以降"sns"と略記）

# カラーマップの設定 (RdBu:赤〜青 ⇔ RdBu_r:青〜赤)
colormap = plt.cm.RdBu_r

# Figureサイズの指定
plt.rcParams['figure.figsize'] = 10, 5

# 相関行列の右上半分を隠したい場合は、次の2行のコメントを解除し、sns.heatmapのオプションに mask=mask を追加
# mask = np.zeros_like(df.corr(), dtype=np.bool)
# mask[np.triu_indices_from(mask)] = True

sns.heatmap(df.corr(numeric_only=True), linewidths=0.1, linecolor='white',   # 相関行列df.corr()を引数としヒートマップ作成
            vmax=1.0, vmin=-1.0, cmap=colormap, annot=True, annot_kws={'fontsize':8}, fmt='.2f')

            # linewidths/linecolor: 格子線の太さ/色
            # cmap:                 カラーマップの指定
            # vmax/vmin:            最大値/最小値
            # annot:                各要素への数値表示
            # annot_kws={'fontsize':8}: 各要素内に表示する数値の文字サイズを指定
            # fmt='.2f':            各要素内に表示する数値の小数点桁数を指定（'.2f'で小数第二位まで）
            # mask:                 可視化から除外する対象（例：mask=mask）
            

## クロス集計（カテゴリー値に有効）
クロス集計・・・2つの変数を「クロス」させて（掛け合わせて）、カテゴリー値の出現値（クラス）ごとの傾向を確認する方法  
`pd.crosstab(行方向に配置する変数, 列方向に配置する変数)`

In [ ]:
### クロス集計 ###
# チケットクラスごとの生存者数を確認
pd.crosstab(df['Pclass'], df['Survived'])  #pd.crosstab(行方向に配置する変数, 列方向に配置する変数)

In [ ]:
### クロス集計 <値正規化バージョン> ###
# チケットクラスごとの生存率を確認
pd.crosstab(df['Pclass'], df['Survived'], normalize='index')  #normalize='index'で行ごとの割合で集計される

In [ ]:
### クロス集計 <応用編> ###
# チケットクラス、生存ごとの平均年齢
pd.crosstab(df['Pclass'], df['Survived'], values=df['Age'], aggfunc='mean')  #values, aggfuncを指定すれば、他列の集計値を出力可能

### 参考：クロス集計表で値の大小に応じて色をつける方法

In [ ]:
# 一旦、クロス集計結果を別のデータフレームに格納
df_cross = pd.crosstab(df['Pclass'], df['Survived'], normalize='index') 

# データフレームのスタイル機能を活用し、行ごとの値の大小に応じてグラデーション
df_cross.style.background_gradient(axis=0, cmap='Reds')
     # axis: 大小を判断する範囲を指定（axis=Noneで全体、axis=0で行ごと、axis=1で列ごと）
     # cmap: カラーマップの指定

参考：カラーマップリスト
https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html

## 欠損値の確認

In [ ]:
df.isnull().sum()

## 欠損値の処理 → 後段の分析時に対応

In [ ]:
# ### 欠損値を中央値で置換 ###
# df['Age'] = df['Age'].fillna(df['Age'].median())  #連続値は中央値／平均値
# df['Embarked'] = df['Embarked'].fillna('S')       #カテゴリー値は最頻値

# Cabinはひとまず除外して分析するため、一旦放置

In [ ]:
# # 欠損値処理後の確認
# df.isnull().sum()

# グラフによる可視化① - ヒストグラム

## 準備（必要なパッケージ、データの読込）

In [ ]:
import matplotlib.pyplot as plt  # グラフ描画パッケージmatplotlib内の、描画設定を行うpyplotモジュールを読込	
import seaborn as sns            # matplotlibをベースにした高機能可視化モジュールsaebornを読込	

## 単一ヒストグラム

In [ ]:
sns.histplot( df['Fare'], kde=False, bins=10 )

## 複数のヒストグラム一括描画

In [ ]:
# pandasパッケージのプロット機能活用
df.hist()

plt.tight_layout()                                 # グラフ同士が重ならないようにするオプション

## 参考：描画設定
※重なりなどの描画崩れや、文字化けなど、描画がうまくいかない場合

In [ ]:
### 以下、デザイン設定 ###
plt.rcParams['figure.figsize'] = 10, 5             # グラフのサイズ指定
plt.rcParams['font.sans-serif'] = ['Hiragino Maru Gothic Pro', 'Yu Gothic', 'Meirio', 'Takao',
                                   'IPAexGothic', 'IPAPGothic', 'VL PGothic', 'Noto Sans CJK JP']
                                                   # 文字化け対策にフォント指定
plt.rcParams['font.size'] = 16                     # フォントサイズ一括指定
plt.tight_layout()                                 # グラフ同士が重ならないようにする
# plt.grid(True)                                   # グリッド表示ON
plt.show()                                         # 各種設定の反映

# グラフによる可視化② - 散布図・箱ひげ図

## 散布図（単一）

In [ ]:
### 散布図（単一） ###	
sns.jointplot(data=df, x='Age', y='Fare')  # dfの「Age」と「Fare」のデータ列の散布図(jointplot)を描画

## 散布図（複数）→ 「散布図行列」

In [ ]:
# 非常に時間がかかるので注意
sns.pairplot(df)

## 生存フラグで色分けした散布図

In [ ]:
# 非常に時間がかかるので注意
sns.pairplot(df, hue= 'Survived' )

## 箱ひげ図

In [ ]:
# pandasパッケージのプロット機能活用
df.boxplot()